**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/as-with).**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [3]:
tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)

taxi_trips


In [4]:
# Write the table name as a string below
table_name = 'taxi_trips'

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
#q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

In [6]:
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

In [7]:
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,285fad7a53021661527cb357220f76c30e605770,455b6b5cae6ca5a17cddd251485f2266d13d6a2c92f07c...,2013-08-02 10:15:00+00:00,2013-08-02 10:30:00+00:00,840,3.40,None,None,14,None,...,0.0,11.05,Cash,None,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
1,2ab4bdd9622c2b80ea5585b0a6eb42db798ece85,8c1af0983bb43246ff0afee514decca8c8a1e5295b7937...,2013-07-18 13:00:00+00:00,2013-07-18 13:30:00+00:00,1080,0.89,None,None,14,None,...,1.0,25.98,Credit Card,None,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
2,60b8903925da004bec6977459b2df831d41aad41,44c4d610e9975f12e4667a42761f382ad05400f7198ece...,2013-08-19 18:00:00+00:00,2013-08-19 18:30:00+00:00,1920,7.40,None,None,14,None,...,6.5,32.58,Credit Card,None,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
3,bc3dd1dda74873c287976a32ec81cee886559a10,b1931345de471dbddb452f82e7d09ba6046580412b1501...,2013-08-21 21:45:00+00:00,2013-08-21 22:00:00+00:00,840,4.00,None,None,14,None,...,0.0,12.05,Cash,None,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None
4,3f93acd82011e28d93be17e6ea9771e744447fe2,1d8661cf286a18a5105e132056c2aadf0b81a551e440fe...,2013-08-27 21:30:00+00:00,2013-08-27 22:30:00+00:00,3600,24.20,None,None,14,None,...,1.0,53.25,Cash,None,41.968069,-87.721559,POINT (-87.7215590627 41.968069),None,None,None


We have some columns with "None", this might be a problem if we want to use them.

After deciding whether you see any important issues, run the code cell below.

In [8]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
You can see the data by calling: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()
```

Some trips in the top few rows have `trip_seconds` or `trip_miles` values of 0. 
Other location fields have values of `None`. That is a problem if we want to use those fields.


### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT EXTRACT(YEAR FROM trip_start_timestamp)</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [9]:
# Your code goes here
rides_per_year_query = """
SELECT COUNT(1) AS num_trips, EXTRACT(year FROM trip_start_timestamp) AS year
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
GROUP BY year
ORDER BY year DESC
"""

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job =  client.query(rides_per_year_query, job_config = safe_config)# Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe()# Your code goes here

# View results
print(rides_per_year_result)

# Check your answer
q_3.check()

   num_trips  year
0    3739963  2020
1   16477365  2019
2   20732088  2018
3   24988003  2017
4   31759339  2016
5   32385875  2015
6   37395436  2014
7   27217716  2013


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [10]:
#q_3.hint()
#q_3.solution()

### 4) Dive slightly deeper

You'd like to take a closer look at rides from 2017.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2017.
2. Modify the query to extract the month rather than the year.

In [11]:
# Your code goes here
rides_per_month_query = """
SELECT EXTRACT(MONTH FROM trip_start_timestamp) AS month, COUNT(1) AS num_trips
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2017
GROUP BY month
ORDER BY month
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config = safe_config) 

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe()

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

    month  num_trips
0       1    1972071
1       2    1909802
2       3    2362105
3       4    2194702
4       5    2323386
5       6    2324472
6       7    2054299
7       8    2079861
8       9    1950631
9      10    2141197
10     11    1907997
11     12    1767480


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [12]:
#q_4.hint()
#q_4.solution()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [13]:
# Your code goes here
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT trip_start_timestamp, trip_miles, trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' and trip_start_timestamp < '2017-07-01' 
                   and trip_seconds > 0 
                   and trip_miles > 0
               )
               SELECT (3600 * SUM(trip_miles) / SUM(trip_seconds)) AS avg_mph, 
               EXTRACT (HOUR from trip_start_timestamp) as hour_of_day, COUNT(1) as num_trips
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config = safe_config) 

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

# Check your answer
q_5.check()

      avg_mph  hour_of_day  num_trips
0   20.230524            0     319339
1   18.937621            1     266529
2   18.777070            2     210147
3   20.158048            3     159668
4   26.736014            4     122183
5   30.769172            5     119312
6   24.588313            6     182738
7   17.735967            7     358406
8   15.079892            8     541775
9   16.543882            9     565548
10  18.539614           10     525120
11  18.928379           11     594603
12  17.838745           12     622324
13  17.671089           13     630181
14  16.974239           14     622465
15  15.688418           15     640430
16  14.283888           16     701435
17  12.462955           17     756627
18  13.646810           18     768251
19  16.642882           19     701064
20  19.536777           20     598614
21  20.433874           21     552726
22  19.531374           22     501095
23  19.877046           23     399587


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the appropriate line below.

In [14]:
#q_5.solution()

That's a hard query. If you made good progress towards the solution, congratulations!

# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the Intro to SQL micro-course.

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*